In [5]:
#from google.colab import drive

#drive.mount('/content/drive')
import pandas as pd
import datetime as dt

#Importing the dataset
df_ = pd.read_csv("/content/drive/MyDrive/DataSets/online_retail_listing.csv",delimiter=';',encoding="latin-1")

df=df_.copy()

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,"6,95",13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,1.12.2009 07:45,"2,1",13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.12.2009 07:45,"1,25",13085.0,United Kingdom


In [7]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      4372
Quantity            0
InvoiceDate         0
Price               0
Customer ID    236682
Country             0
dtype: int64

In [15]:

df["Invoice"].unique()
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,1048575.0,9.957525,133.518692,-74215.00,1.00,3.0,10.00,74215.0
Price,1048575.0,4.627346,122.802412,-53594.36,1.25,2.1,4.15,38970.0
Customer ID,811893.0,15324.712265,1697.033034,12346.00,13971.00,15260.0,16795.00,18287.0
TotalPrice,1048575.0,18.093596,179.889689,-77183.60,3.75,9.9,17.70,77183.6


In [23]:
#Calculating Metrics

# Customer purchased products at last time
df["InvoiceDate"].max()

df.columns.values

array(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'TotalPrice'], dtype=object)

In [18]:
ref_date= pd.datetime(2011, 12, 12)

<ipython-input-18-b5dd5df1f992>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  ref_date= pd.datetime(2011, 12, 12)


In [24]:
# Calculate recency, frequency, and monetary value for each customer
rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda date: (ref_date - date.max()).days,
                                     "Invoice": lambda Invoice: Invoice.nunique(),
                                     "TotalPrice":lambda TotalPrice: TotalPrice.sum()})

rfm.columns = ["Recency", "Frequency", "Monetary"]

rfm = rfm[rfm["Monetary"] > 0]

rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12347.0,41,7,5408.50
12348.0,77,5,2019.40
12349.0,20,5,4404.54
12350.0,312,1,334.40
12351.0,377,1,300.93


In [25]:
#Calculating RFM Scores
rfm["Recency_score"] = pd.cut(rfm["Recency"], 5, labels=[5,4,3,2,1])

rfm["Monetary_score"] = pd.cut(rfm["Monetary"], 5, labels=[1,2,3,4,5])

rfm["Frequency_score"] = pd.cut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])

rfm["RF_SCORE"] = (rfm["Recency_score"].astype(str) + rfm["Frequency_score"].astype(str))

rfm.head(10)

,Recency,Frequency,Monetary,Recency_score,Monetary_score,Frequency_score,RF_SCORE
Customer ID,,,,,,,
12347.0,41,7,5408.50,5,1,4,54
12348.0,77,5,2019.40,5,1,3,53
12349.0,20,5,4404.54,5,1,3,53
12350.0,312,1,334.40,4,1,1,41
12351.0,377,1,300.93,4,1,1,41
12352.0,74,13,1889.21,5,1,5,55
12353.0,206,2,406.76,5,1,2,52
12354.0,234,1,1079.40,4,1,1,41
12355.0,97,2,947.61,5,1,2,52


In [26]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm["Segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)

rfm.head(10)

,Recency,Frequency,Monetary,Recency_score,Monetary_score,Frequency_score,RF_SCORE,Segment
Customer ID,,,,,,,,
12347.0,41,7,5408.50,5,1,4,54,champions
12348.0,77,5,2019.40,5,1,3,53,potential_loyalists
12349.0,20,5,4404.54,5,1,3,53,potential_loyalists
12350.0,312,1,334.40,4,1,1,41,promising
12351.0,377,1,300.93,4,1,1,41,promising
12352.0,74,13,1889.21,5,1,5,55,champions
12353.0,206,2,406.76,5,1,2,52,potential_loyalists
12354.0,234,1,1079.40,4,1,1,41,promising
12355.0,97,2,947.61,5,1,2,52,potential_loyalists


In [29]:
#Writing to CSV
rfm.to_csv('RFM Analysis.csv', encoding='utf-8')